# Project B: Knowledge Distillation for Building Lightweight Deep Learning Models in Visual Classification Tasks

In [4]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from typing import Union

tf.enable_v2_behavior()

BATCH_SIZE = 32
NUM_EPOCHS = 10

# Data loading

In [11]:
import pandas as pd

# Load train and test splits.
anno_dir = r'D:\Github Repos\ECE1512\-ECE1512_2022W_ProjectRepo_StephanieDiNunzio\Project B\Project_B_Supp\mhist_dataset\annotations.csv' #you should change to your directory
df = pd.read_csv(anno_dir)
df = df[['Image Name','Majority Vote Label','Partition']]
df

,Image Name,Majority Vote Label,Partition
0,MHIST_aaa.png,SSA,train
1,MHIST_aab.png,HP,train
2,MHIST_aac.png,SSA,train
3,MHIST_aae.png,HP,train
4,MHIST_aaf.png,SSA,train
...,...,...,...
3147,MHIST_cpn.png,SSA,train
3148,MHIST_cfc.png,SSA,test
3149,MHIST_cgp.png,SSA,test
3150,MHIST_dlf.png,SSA,train


In [15]:
import shutil

hp_test_dir = r'D:\Github Repos\ECE1512\-ECE1512_2022W_ProjectRepo_StephanieDiNunzio\Project B\Project_B_Supp\mhist_dataset\images_sorted\test\HP_test'
hp_train_dir = r'D:\Github Repos\ECE1512\-ECE1512_2022W_ProjectRepo_StephanieDiNunzio\Project B\Project_B_Supp\mhist_dataset\images_sorted\train\HP_train'
ssa_test_dir = r'D:\Github Repos\ECE1512\-ECE1512_2022W_ProjectRepo_StephanieDiNunzio\Project B\Project_B_Supp\mhist_dataset\images_sorted\test\SSA_test'
ssa_train_dir = r'D:\Github Repos\ECE1512\-ECE1512_2022W_ProjectRepo_StephanieDiNunzio\Project B\Project_B_Supp\mhist_dataset\images_sorted\train\SSA_train'

img_root_dir = r'D:\Github Repos\ECE1512\-ECE1512_2022W_ProjectRepo_StephanieDiNunzio\Project B\Project_B_Supp\mhist_dataset\images\images'

for i, row in df.iterrows():
    img_dir = img_root_dir + '\\' + row['Image Name']
    if row['Majority Vote Label'] == 'SSA':
        if row['Partition'] == 'train':
            new_dir = ssa_train_dir + '\\' + row['Image Name']
            shutil.copyfile(img_dir,new_dir)
        else:
            new_dir = ssa_test_dir + '\\' + row['Image Name']
            shutil.copyfile(img_dir,new_dir)
    else:
        if row['Partition'] == 'train':
            new_dir = hp_train_dir + '\\' + row['Image Name']
            shutil.copyfile(img_dir,new_dir)
        else:
            new_dir = hp_test_dir + '\\' + row['Image Name']
            shutil.copyfile(img_dir,new_dir)
    #break

In [5]:
from keras.preprocessing.image import ImageDataGenerator

test_dir = r'D:\Github Repos\ECE1512\-ECE1512_2022W_ProjectRepo_StephanieDiNunzio\Project B\Project_B_Supp\mhist_dataset\images_sorted\test'
train_dir = r'D:\Github Repos\ECE1512\-ECE1512_2022W_ProjectRepo_StephanieDiNunzio\Project B\Project_B_Supp\mhist_dataset\images_sorted\train'

train_datagen = ImageDataGenerator(rescale=1/255.,
shear_range=0.1,
rotation_range=15,
zoom_range=0.2, # zoom
horizontal_flip=True,
vertical_flip=True)

test_datagen = ImageDataGenerator(rescale=1/255.)

train_generator = train_datagen.flow_from_directory(train_dir,
class_mode='binary',
interpolation='bilinear',
target_size=(224, 224),
batch_size=32,
shuffle=True,)

test_generator = test_datagen.flow_from_directory(test_dir,
class_mode='binary',
interpolation='bilinear',
target_size=(224, 224),
batch_size=32,
shuffle=False)

Found 2175 images belonging to 2 classes.
Found 977 images belonging to 2 classes.


In [3]:
import math


# Model creation

In [16]:
import keras
from tensorflow.keras import layers

resnet = tf.keras.applications.resnet_v2.ResNet50V2(weights='imagenet', include_top=False)
resnet.trainable = False

inputs = keras.Input(shape=(224,224,3))
x = resnet(inputs, training=False)
x= layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(1)(x)


teacher_model = keras.Model(inputs, outputs)
teacher_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50v2 (Functional)      (None, None, None, 2048)  23564800  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 2049      
Total params: 23,566,849
Trainable params: 2,049
Non-trainable params: 23,564,800
_________________________________________________________________


# Training models

In [17]:
LEARNING_RATE = 0.001
teacher_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [18]:
teacher_model.fit(train_generator,
                    steps_per_epoch=train_generator.n // BATCH_SIZE,
                    epochs=10,                  
                    shuffle = True,
                    verbose=1,
                    validation_data=test_generator,
                    validation_steps=test_generator.n // BATCH_SIZE,)

Epoch 1/10
67/67 [==============================] - 18s 239ms/step - loss: 0.5261 - accuracy: 0.7345 - val_loss: 0.5046 - val_accuracy: 0.7146
Epoch 2/10
67/67 [==============================] - 15s 227ms/step - loss: 0.4266 - accuracy: 0.7928 - val_loss: 0.4582 - val_accuracy: 0.7542
Epoch 3/10
67/67 [==============================] - 15s 227ms/step - loss: 0.4081 - accuracy: 0.8021 - val_loss: 0.4528 - val_accuracy: 0.7604
Epoch 4/10
67/67 [==============================] - 15s 226ms/step - loss: 0.3869 - accuracy: 0.8101 - val_loss: 0.4441 - val_accuracy: 0.7729
Epoch 5/10
67/67 [==============================] - 15s 227ms/step - loss: 0.3920 - accuracy: 0.8115 - val_loss: 0.4482 - val_accuracy: 0.7615
Epoch 6/10
67/67 [==============================] - 15s 229ms/step - loss: 0.3798 - accuracy: 0.8171 - val_loss: 0.4536 - val_accuracy: 0.7615
Epoch 7/10
67/67 [==============================] - 15s 229ms/step - loss: 0.3774 - accuracy: 0.8245 - val_loss: 0.4341 - val_accuracy: 0.7833

In [218]:
teacher_model.save(r'D:\Github Repos\ECE1512\-ECE1512_2022W_ProjectRepo_StephanieDiNunzio\Project B\Project_B_Supp\models\mhist_model_init.h5')

In [19]:
resnet.trainable = True

LEARNING_RATE = 0.0001
teacher_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [20]:
teacher_model.fit(train_generator,
                    steps_per_epoch=train_generator.n // BATCH_SIZE,
                    epochs=25,                  
                    shuffle = True,
                    verbose=1,
                    validation_data=test_generator,
                    validation_steps=test_generator.n // BATCH_SIZE,)

Epoch 1/25
67/67 [==============================] - 22s 258ms/step - loss: 0.6173 - accuracy: 0.7070 - val_loss: 0.5658 - val_accuracy: 0.6427
Epoch 2/25
67/67 [==============================] - 16s 231ms/step - loss: 0.5466 - accuracy: 0.7158 - val_loss: 0.5588 - val_accuracy: 0.6427
Epoch 3/25
67/67 [==============================] - 16s 236ms/step - loss: 0.4883 - accuracy: 0.7280 - val_loss: 0.5106 - val_accuracy: 0.7042
Epoch 4/25
67/67 [==============================] - 16s 231ms/step - loss: 0.4668 - accuracy: 0.7625 - val_loss: 0.4893 - val_accuracy: 0.7625
Epoch 5/25
67/67 [==============================] - 16s 230ms/step - loss: 0.4316 - accuracy: 0.7797 - val_loss: 0.4381 - val_accuracy: 0.7896
Epoch 6/25
67/67 [==============================] - 15s 229ms/step - loss: 0.4189 - accuracy: 0.7839 - val_loss: 0.4183 - val_accuracy: 0.7833
Epoch 7/25
67/67 [==============================] - 16s 236ms/step - loss: 0.3710 - accuracy: 0.8255 - val_loss: 0.3807 - val_accuracy: 0.7958

In [38]:
teacher_model.save(r'D:\Github Repos\ECE1512\-ECE1512_2022W_ProjectRepo_StephanieDiNunzio\Project B\Project_B_Supp\models\mhist_model.h5')

C:\Users\Stephanie\anaconda3\envs\ece1512\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [39]:
print("Evaluate on test data")
results = teacher_model.evaluate_generator(generator=test_generator)
print("test loss, test acc:", results)

Evaluate on test data


C:\Users\Stephanie\anaconda3\envs\ece1512\lib\site-packages\keras\engine\training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


test loss, test acc: [0.3987638056278229, 0.838280439376831]


In [67]:
from sklearn import metrics
import numpy as np
import pandas as pd

test_data = []
test_labels = []

for j in range(len(test_generator)):
    test_labels += test_generator[j][1].tolist()

from sklearn.metrics import classification_report
pred = teacher_model.predict_generator(test_generator)
pred_classes = []



for i in pred:
    if i[0] < 0:
        pred_classes.append(0)
    else:
        pred_classes.append(1)

report = classification_report(test_labels,pred_classes,output_dict=True)
df = pd.DataFrame(report).transpose()
df.pop('support')
df = pd.concat([df[0:8] , df[10:]])
df

C:\Users\Stephanie\anaconda3\envs\ece1512\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


,precision,recall,f1-score
0.0,0.835655,0.972447,0.898876
1.0,0.934363,0.672222,0.781906
accuracy,0.861822,0.861822,0.861822
macro avg,0.885009,0.822335,0.840391
weighted avg,0.872026,0.861822,0.855776


In [27]:


# Build student. NO KD

mobilenet = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(224,224,3), include_top=False, weights='imagenet')
mobilenet.trainable = False

inputs2 = keras.Input(shape=(224,224,3))
y = mobilenet(inputs2, training=False)
y= layers.GlobalAveragePooling2D()(y)
outputs2 = layers.Dense(1)(y)

student_model = keras.Model(inputs2, outputs2)
student_model.summary()


# 

LEARNING_RATE = 0.001
student_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_4 ( (None, 1280)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,281
Non-trainable params: 2,257,984
_________________________________________________________________


In [28]:
student_model.fit(train_generator,
                    steps_per_epoch=train_generator.n // BATCH_SIZE,
                    epochs=10,                  
                    shuffle = True,
                    verbose=1,
                    validation_data=test_generator,
                    validation_steps=test_generator.n // BATCH_SIZE,)

Epoch 1/10
67/67 [==============================] - 18s 237ms/step - loss: 0.5764 - accuracy: 0.7224 - val_loss: 0.5204 - val_accuracy: 0.6958
Epoch 2/10
67/67 [==============================] - 15s 231ms/step - loss: 0.4575 - accuracy: 0.7713 - val_loss: 0.5021 - val_accuracy: 0.7115
Epoch 3/10
67/67 [==============================] - 15s 227ms/step - loss: 0.4241 - accuracy: 0.7886 - val_loss: 0.4954 - val_accuracy: 0.7250
Epoch 4/10
67/67 [==============================] - 15s 228ms/step - loss: 0.4121 - accuracy: 0.8059 - val_loss: 0.4721 - val_accuracy: 0.7448
Epoch 5/10
67/67 [==============================] - 15s 228ms/step - loss: 0.3966 - accuracy: 0.8045 - val_loss: 0.4785 - val_accuracy: 0.7437
Epoch 6/10
67/67 [==============================] - 16s 232ms/step - loss: 0.3857 - accuracy: 0.8199 - val_loss: 0.4835 - val_accuracy: 0.7417
Epoch 7/10
67/67 [==============================] - 15s 230ms/step - loss: 0.3867 - accuracy: 0.8077 - val_loss: 0.4843 - val_accuracy: 0.7469

In [208]:
student_model.save(r'D:\Github Repos\ECE1512\-ECE1512_2022W_ProjectRepo_StephanieDiNunzio\Project B\Project_B_Supp\models\mhist_model_stu_init.h5')

C:\Users\Stephanie\anaconda3\envs\ece1512\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [29]:
mobilenet.trainable = True

LEARNING_RATE = 0.0001
student_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [30]:
student_model.fit(train_generator,
                    steps_per_epoch=train_generator.n // BATCH_SIZE,
                    epochs=25,                  
                    shuffle = True,
                    verbose=1,
                    validation_data=test_generator,
                    validation_steps=test_generator.n // BATCH_SIZE)

Epoch 1/25
67/67 [==============================] - 19s 252ms/step - loss: 0.5663 - accuracy: 0.7102 - val_loss: 0.5835 - val_accuracy: 0.6479
Epoch 2/25
67/67 [==============================] - 16s 241ms/step - loss: 0.4684 - accuracy: 0.7476 - val_loss: 0.5031 - val_accuracy: 0.6458
Epoch 3/25
67/67 [==============================] - 16s 240ms/step - loss: 0.4676 - accuracy: 0.7490 - val_loss: 0.5282 - val_accuracy: 0.6427
Epoch 4/25
67/67 [==============================] - 16s 244ms/step - loss: 0.4348 - accuracy: 0.7569 - val_loss: 0.5074 - val_accuracy: 0.6990
Epoch 5/25
67/67 [==============================] - 16s 238ms/step - loss: 0.3702 - accuracy: 0.8190 - val_loss: 0.5557 - val_accuracy: 0.7125
Epoch 6/25
67/67 [==============================] - 16s 237ms/step - loss: 0.3605 - accuracy: 0.8259 - val_loss: 0.4858 - val_accuracy: 0.7906
Epoch 7/25
67/67 [==============================] - 16s 237ms/step - loss: 0.3174 - accuracy: 0.8507 - val_loss: 0.6043 - val_accuracy: 0.7812

In [31]:
student_model.save(r'D:\Github Repos\ECE1512\-ECE1512_2022W_ProjectRepo_StephanieDiNunzio\Project B\Project_B_Supp\models\mhist_model_stu_fine.h5')

In [68]:
student_model = keras.models.load_model(r'D:\Github Repos\ECE1512\-ECE1512_2022W_ProjectRepo_StephanieDiNunzio\Project B\Project_B_Supp\models\mhist_model_stu_fine.h5')

In [69]:
test_data = []
test_labels = []

for j in range(len(test_generator)):
    test_labels += test_generator[j][1].tolist()

from sklearn.metrics import classification_report
pred = student_model.predict_generator(test_generator)
pred_classes = []



for i in pred:
    if i[0] < 0:
        pred_classes.append(0)
    else:
        pred_classes.append(1)

report = classification_report(test_labels,pred_classes,output_dict=True)
df = pd.DataFrame(report).transpose()
df.pop('support')
df = pd.concat([df[0:8] , df[10:]])
df

C:\Users\Stephanie\anaconda3\envs\ece1512\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


,precision,recall,f1-score
0.0,0.790257,0.946515,0.861357
1.0,0.861345,0.569444,0.685619
accuracy,0.807574,0.807574,0.807574
macro avg,0.825801,0.757980,0.773488
weighted avg,0.816451,0.807574,0.796602


# Student loss function

In [36]:


def distillation_loss(teacher_logits: tf.Tensor, student_logits: tf.Tensor,
                      temperature: Union[float, tf.Tensor]):
  """Compute distillation loss.

  This function computes cross entropy between softened logits and softened
  targets. The resulting loss is scaled by the squared temperature so that
  the gradient magnitude remains approximately constant as the temperature is
  changed. For reference, see Hinton et al., 2014, "Distilling the knowledge in
  a neural network."

  Args:
    teacher_logits: A Tensor of logits provided by the teacher.
    student_logits: A Tensor of logits provided by the student, of the same
      shape as `teacher_logits`.
    temperature: Temperature to use for distillation.

  Returns:
    A scalar Tensor containing the distillation loss.
  """
 # your code start from here for step 3
  soft_targets = tf.nn.sigmoid(teacher_logits)

  return tf.reduce_mean(
      tf.nn.sigmoid_cross_entropy_with_logits(
          soft_targets, student_logits / temperature)) * temperature ** 2

def compute_student_loss(images, labels):
  """Compute class knowledge distillation student loss for given images
     and labels.

  Args:
    images: Tensor representing a batch of images.
    labels: Tensor representing a batch of labels.

  Returns:
    Scalar loss Tensor.
  """
  student_class_logits = student_model(images, training=True)

  # Compute class distillation loss between student class logits and
  # softened teacher class targets probabilities.

  # your code start from here for step 3

  teacher_class_logits = teacher_model(images, training=False)
  distillation_loss_value = distillation_loss(teacher_class_logits, student_class_logits, DISTILLATION_TEMPERATURE)

  # Compute cross-entropy loss with hard targets.

  # your code start from here for step 3
  loss = keras.losses.BinaryCrossentropy(from_logits=True)
  
  cross_entropy_loss_value = loss(labels,student_class_logits).numpy()

  return ALPHA*cross_entropy_loss_value + (1-ALPHA)*distillation_loss_value

In [ ]:
teacher_model = keras.models.load_model(r'D:\Github Repos\ECE1512\-ECE1512_2022W_ProjectRepo_StephanieDiNunzio\Project B\Project_B_Supp\models\mhist_model.h5')

In [32]:
print("Evaluate on test data")
results = student_model.evaluate_generator(generator=test_generator)
print("test loss, test acc:", results)

Evaluate on test data
test loss, test acc: [0.6105268001556396, 0.8034800291061401]


In [70]:
from keras_flops import get_flops

flops_teacher = get_flops(teacher_model, batch_size=BATCH_SIZE)
print(flops_teacher)
flops_student = get_flops(student_model, batch_size=BATCH_SIZE)
print(flops_student)

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
223623861408
19604148192


In [41]:
# Hyperparameters for distillation (need to be tuned).
ALPHA = 0.5 # task balance between cross-entropy and distillation loss
DISTILLATION_TEMPERATURE = 2.5 #temperature hyperparameter


# Build student.

mobilenet2 = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(224,224,3), weights='imagenet', include_top=False)
mobilenet2.trainable = False

inputs3 = keras.Input(shape=(224,224,3))
z = mobilenet2(inputs3, training=False)
z= layers.GlobalAveragePooling2D()(z)
outputs3 = layers.Dense(1)(z)

student_model_KD = keras.Model(inputs3, outputs3)
student_model_KD.summary()


# 

LEARNING_RATE = 0.001
student_model_KD.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=compute_student_loss,
              metrics=['accuracy'])

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_10  (None, 1280)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,281
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
student_model_KD.fit(train_generator,
                    steps_per_epoch=train_generator.n // BATCH_SIZE,
                    epochs=25,                  
                    shuffle = True,
                    verbose=1,
                    validation_data=test_generator,
                    validation_steps=test_generator.n // BATCH_SIZE)# Student loss function
